In [3]:
!pip install deap

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.4/135.4 kB 2.4 MB/s eta 0:00:00


In [4]:
import random
import numpy as np
from deap import base, creator, tools

# Población original
poblacion_original = [10, 13, 3, 4, 8, 1, 12, 5, 7, 6, 9, 11]

# Configuración de DEAP
creator.create("FitnessMax", base.Fitness, weights=(1.0,))  # Maximizar
creator.create("Individual", list, fitness=creator.FitnessMax)

toolbox = base.Toolbox()

# Función para convertir a binario (para mostrar resultados)
def to_binary(num):
    return format(num, '08b')  # 8 bits

# Definir el atributo individual (en binario, de 8 bits)
toolbox.register("attr_int", lambda: random.choice(poblacion_original))

# Crear un individuo como una lista de enteros (de longitud 1, ya que son números individuales)
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_int, n=1)

# Crear una población de individuos
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

# Evaluación: utilizamos el número en sí mismo como fitness
def eval_individual(individual):
    return individual[0],  # Necesita devolver una tupla

toolbox.register("evaluate", eval_individual)

# Función para hacer el cruce (1 punto de cruce)
def custom_crossover(parent1, parent2, punto_cruce):
    bin1 = to_binary(parent1[0])
    bin2 = to_binary(parent2[0])

    # Cruzamos en el punto especificado
    child1_bin = bin1[:punto_cruce] + bin2[punto_cruce:]
    child2_bin = bin2[:punto_cruce] + bin1[punto_cruce:]

    # Convertimos de binario a decimal
    child1 = int(child1_bin, 2)
    child2 = int(child2_bin, 2)

    return creator.Individual([child1]), creator.Individual([child2])

# Función para hacer la mutación (mutamos un bit específico)
def custom_mutation(individual, punto_mutacion):
    bin_ind = to_binary(individual[0])

    # Realizamos la mutación en el punto especificado
    if bin_ind[punto_mutacion] == '0':
        new_bin_ind = bin_ind[:punto_mutacion] + '1' + bin_ind[punto_mutacion + 1:]
    else:
        new_bin_ind = bin_ind[:punto_mutacion] + '0' + bin_ind[punto_mutacion + 1:]

    # Convertimos de nuevo a decimal
    individual[0] = int(new_bin_ind, 2)

    return individual,

# Registrar las funciones de cruce y mutación
toolbox.register("mate", custom_crossover)
toolbox.register("mutate", custom_mutation)

# Algoritmo genético
def ejecutar_generaciones(poblacion_inicial, generaciones):
    poblacion = toolbox.population(n=len(poblacion_inicial))

    for gen in range(generaciones):
        print(f"\n--- Generación {gen + 1} ---")

        punto_cruce = int(input(f"Introduce el punto de cruce para la generación {gen + 1} (0-7): "))
        punto_mutacion = int(input(f"Introduce el bit de mutación (0-7) para la generación {gen + 1}: "))

        # Seleccionar parejas para cruzar
        offspring = []
        for i in range(0, len(poblacion), 2):
            parent1 = poblacion[i]
            parent2 = poblacion[i + 1] if i + 1 < len(poblacion) else poblacion[0]

            child1, child2 = toolbox.mate(parent1, parent2, punto_cruce)
            offspring.append(child1)
            offspring.append(child2)

        # Aplicar mutación
        for individual in offspring:
            if random.random() < 0.5:  # Probabilidad de mutación
                toolbox.mutate(individual, punto_mutacion)

        # Evaluar nueva generación
        poblacion[:] = offspring
        fitness_values = list(map(toolbox.evaluate, poblacion))
        print("Nueva población (decimal):", [x[0] for x in fitness_values])

    return poblacion

# Parámetros iniciales
generaciones = int(input("Introduce el número de generaciones: "))

# Ejecutar el algoritmo
poblacion_final = ejecutar_generaciones(poblacion_original, generaciones)


Introduce el número de generaciones: 3

--- Generación 1 ---
Introduce el punto de cruce para la generación 1 (0-7): 4
Introduce el bit de mutación (0-7) para la generación 1: 6
Nueva población (decimal): [4, 12, 11, 1, 6, 13, 1, 1, 11, 4, 6, 5]

--- Generación 2 ---
Introduce el punto de cruce para la generación 2 (0-7): 5
Introduce el bit de mutación (0-7) para la generación 2: 7
Nueva población (decimal): [4, 13, 9, 3, 5, 14, 0, 1, 13, 3, 4, 6]

--- Generación 3 ---
Introduce el punto de cruce para la generación 3 (0-7): 3
Introduce el bit de mutación (0-7) para la generación 3: 5
Nueva población (decimal): [13, 4, 7, 13, 10, 1, 1, 4, 3, 9, 2, 4]
